<a href="https://colab.research.google.com/github/enverakbacak/EMIR/blob/main/myGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

In [ ]:
cd drive/MyDrive/ColabNotebooks/generators/

In [9]:
#coding=utf-8
import tensorflow as tf
import cv2
import glob
import numpy as np
import sys,os
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import time
import glob
import os, os.path
import re
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Input
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import pandas as pd  
import matplotlib.pyplot as plt
from skimage.transform import resize   # for resizing images
from tensorflow.keras.optimizers import SGD
import matplotlib.image as mpimg

In [16]:
def generator(samples, labels, batch_size=32):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    num_samples = len(samples)
 
    while True: # Loop forever so the generator never terminates
      
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]
            label_samples = labels[offset:offset+batch_size]
             
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = [] 
            # For each example
            for batch_sample in batch_samples:
              filename = 'Lamda/images/' + batch_sample[0] + '.jpg'
              image = plt.imread(filename)
              # Read label (y)
              X_train.append(image)
            X_train = np.array(X_train)
            #from tensorflow.keras.applications.vgg16 import preprocess_input
            #X_train = preprocess_input(X_train) # preprocessing the input data
            
            for label_sample in label_samples:
              y_train.append(label_sample) 
            y_train = np.array(y_train)            
            

            # The generator-y part: yield the next training batch
            yield X_train, y_train
    

# Import list of train and validation data (image filenames and image labels)
# Note this is not valid code.

In [20]:
data = pd.read_csv('data.csv')
print(len(data))
labels = pd.read_csv('Y.csv')
labels.shape
print(labels.shape[:])

labels = np.asarray(labels).astype('float32').reshape((-1,1))
print(labels.shape[:])

# Create generator
train_generator = generator(data, labels, batch_size=32)


1999
(1999, 5)
(9995, 1)


In [21]:
image_size = 229
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
base_model.trainable = False

batch_size = 32
hash_bits = 16
inputs    = tensorflow.keras.Input(shape = (image_size, image_size,3))
x         = base_model(inputs)
#flatten   = Flatten()(x)
Dense_1   = Dense(1024)(x)
avgPool = GlobalAveragePooling2D()(Dense_1)
Dense_2 = Dense(hash_bits ,activation='tanh')(avgPool)
Dense_3 = Dense(5, activation='sigmoid')(Dense_2)
model = Model(inputs, Dense_3)
print(model.summary())



sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)


def c_loss_1(y_true, y_pred):
    return  tf.keras.losses.binary_crossentropy(y_true, y_pred)

def c_loss_2(noise_1, noise_2):
    noise_1 = tf.cast(Dense_2 > 0.5 , tf.float32 )
    noise_2 = Dense_2 
    return  tf.keras.losses.binary_crossentropy(noise_1, noise_2)
model.compile(loss = [c_loss_1, c_loss_2],  optimizer=sgd, metrics=['accuracy'],)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 229, 229, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
dense_6 (Dense)              (None, 5, 5, 1024)        2098176   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                16400     
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 85        
Total params: 23,917,445
Trainable params: 2,114,661
Non-trainable params: 21,802,784
_______________________________________

In [ ]:
model.fit_generator(train_generator, verbose=1, steps_per_epoch = 2000/32, epochs=32)